In [8]:
########################################################################################################
## Ejecución de Pig en Jupyter
##    A continuación se describe como ejecutar comandos de Pig en Jupyter usando la extensión de Jupyter bigdata.
%load_ext bigdata
%pig_start
%timeout 300
## WordCount en Apache Pig
##   Se cargan los archivos en Apache Pig.
!hadoop fs -put *.csv .
!hadoop fs -ls

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata
put: `truck_event_text_partition.csv': File exists
Found 7 items
-rw-r--r--   1 root supergroup        244 2020-02-28 22:44 data.tsv
-rw-r--r--   1 root supergroup         56 2020-02-28 22:40 data0.tsv
-rw-r--r--   1 root supergroup         56 2020-02-28 22:40 data1.tsv
-rw-r--r--   1 root supergroup         56 2020-02-28 22:44 datatest.tsv
drwxr-xr-x   - root supergroup          0 2020-02-29 00:50 output
drwxr-xr-x   - root supergroup          0 2020-02-29 00:09 q01-10
-rw-r--r--   1 root supergroup    2271958 2020-02-29 00:56 truck_event_text_partition.csv


In [14]:
%%pig
--
-- Carga el archivo desde el disco duro
--
dataTable = LOAD 'truck_event_text_partition.csv' USING PigStorage(',')
    AS (driverId:INT,
        truckId:INT,
        eventTime:CHARARRAY,
        eventType:CHARARRAY,
        longitude:DOUBLE,
        latitude:DOUBLE,
        eventKey:CHARARRAY,
        correlationId:CHARARRAY,
        driverName:CHARARRAY,
        routeId:FLOAT,
        routeName:CHARARRAY,
        eventDate:CHARARRAY);

 --
 -- Carga el archivo desde el disco duro
 --
 dataTable = LOAD 'truck_event_text_partition.csv' USING PigStorage(',')
    AS (driverId:INT,
        truckId:INT,
        eventTime:CHARARRAY,
        eventType:CHARARRAY,
        longitude:DOUBLE,
        latitude:DOUBLE,
        eventKey:CHARARRAY,
        correlationId:CHARARRAY,
        driverName:CHARARRAY,
        routeId:FLOAT,
        routeName:CHARARRAY,
        eventDate:CHARARRAY);


In [18]:
%%pig
x = FOREACH dataTable GENERATE driverId,truckId,eventTime;
y = LIMIT x 10;
z = ORDER y BY $0, $1, $2;
DUMP z;

 x = FOREACH dataTable GENERATE driverId,truckId,eventTime;
 y = LIMIT x 10;
 z = ORDER y BY $0, $1, $2;
 DUMP z;
2020-02-29 01:13:46,332 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-29 01:13:46,407 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-29 01:13:46,417 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2020-02-29 01:13:46,427 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2020-02-29 01:13:46,450 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2020-02-29 01:13:46,868 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for job: job_1582928020258_0077
2020-02-29 01:13:46,870 [JobControl] INFO  org.apache.hadoop.mapred.YARNRunner - J

In [19]:
%pig_quit

In [20]:
########################################################################################################
########################################################################################################
########################################################################################################
########################################################################################################
########################################################################################################
########################################################################################################
## Código en Apache Pig
##   Nota. Se usan los dos guiones -- para comentario de una línea y /* … */ para comentarios de varias líneas.
########################################################################################################
########################################################################################################
########################################################################################################
########################################################################################################
########################################################################################################
########################################################################################################

In [27]:
## Limpieza del HDFS y de la máquina local
## Se crea el directorio de entrada
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rmdir output input
## Se elimina la carpeta si existe
!rm -rf output input
# !mkdir output input

rm: `input/*': No such file or directory
Deleted output/_SUCCESS
Deleted output/part-r-00000
rmdir: `input': No such file or directory


In [28]:
%%writefile script.pig

-- carga de datos
dataTable = LOAD 'truck_event_text_partition.csv' USING PigStorage(',')
    AS (driverId:INT,
        truckId:INT,
        eventTime:CHARARRAY,
        eventType:CHARARRAY,
        longitude:DOUBLE,
        latitude:DOUBLE,
        eventKey:CHARARRAY,
        correlationId:CHARARRAY,
        driverName:CHARARRAY,
        routeId:FLOAT,
        routeName:CHARARRAY,
        eventDate:CHARARRAY);
    
-- filtrar y ordenar
x = FOREACH dataTable GENERATE driverId,truckId,eventTime;
y = LIMIT x 10;
z = ORDER y BY $0, $1, $2;

-- escribe el archivo de salida
STORE z INTO 'output' USING PigStorage(',');

-- copia los archivos del HDFS al sistema local
fs -get output/ .

Overwriting script.pig


In [29]:
## Ejecución del script en modo batch
!pig -execute 'run script.pig'

2020-02-29 01:26:56,744 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2020-02-29 01:26:58,066 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.textoutputformat.separator is deprecated. Instead, use mapreduce.output.textoutputformat.separator
2020-02-29 01:26:58,434 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-29 01:26:58,608 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2020-02-29 01:26:58,611 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.output.compress is deprecated. Instead, use mapreduce.output.fileoutputformat.compress
2020-02-29 01:26:58,614 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.reduce.tasks is deprecated. Instead, use

In [30]:
## Visualización de los resultados en el HDFS
!hadoop fs -cat output/*

11,74,59:21.7
11,74,59:22.5
14,25,59:21.4
18,16,59:21.7
20,41,59:22.5
22,87,59:21.7
22,87,59:22.3
23,68,59:22.4
27,105,59:21.7
32,42,59:22.5


In [31]:
## Visualilzación de resultados en el sistema local
!cat output/*

11,74,59:21.7
11,74,59:22.5
14,25,59:21.4
18,16,59:21.7
20,41,59:22.5
22,87,59:21.7
22,87,59:22.3
23,68,59:22.4
27,105,59:21.7
32,42,59:22.5
